In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import random
import os
from IPython.display import display
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
import tensorflow_addons as tfa
import decimal
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import math
from tensorflow.keras import activations

In [8]:
sessions_csv = "/Volumes/Seagate/natasha-diploma/sessions1.csv"
sessions_df = pd.read_csv(sessions_csv, index_col=None, header=0)
sessions_df.drop(sessions_df.columns[sessions_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(sessions_df)

,0,1,2,3,4,5,6,7,8,9
0,403a42b604f504069c41d39c5a4fc2,0dbe493275a25285829b979316d566,9aecf32a60e4e0f7f967caecb6c798,23ee5640ec588ffaa78afa3268b04a,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,26b3f811a6b9ed75d3cc5198105fa4,06132809afbe1c781e1b9605452081,67354be5f6f02a74e7661f9529daeb
1,d66440e7524c0a8b9b206fbb7295ba,a6d83ff19ed58d43644ca319a2c5d4,a98af02d324af4f43b7a42c82b73d0,62e8a540d2321f61f2b47dd26103e4,807d308f6b0d8df70b5ffbd5b88f38,7f778c07837b7646904a091a481145,76dc726c7fb950a0fff79073b229f6,930e6d99ffcb060d6b030d4cc90680,4b7669ea29a6770d7ee65e57b34e63,44befb98302cfa436627fa3af38642
2,231b80c884b2f1390e17f09a187889,bd6707bb04cfb8880a5f783c85be20,d6159553e41857fe23df9d3e746284,c0314a446fbef65ce9626b037a70da,961c336b0c2fa04c883c639dfee1ba,514f3ee1f00df05dc3900251972245,2c72ca8794dd4eb8f5ae90ca0dce24,c292588c17f2fb55db17a98774ef07,40fa578cd222749786f5aa816ad3c5,7e2582b83c27340f388fdce46a9209
3,d50faca942c93b02863dc4cc86a839,2530d7a6ecf471f695a2ef7a8a4191,f3153c49fddebe52085185876b61c5,914880359e59dbe7b7e60c4ddc82f4,4247088ee2239f9885a836fe4735cb,9914764b34cff2aac25a82a33d5494,6fcff414343456465369b05f6851eb,41b772a5b18bc724562bfb60f4f3ac,34cfd44ad5cc2d63441ebf437eccb5,e291f8ae1de4a38c71027e9c91d51c
4,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,23ee5640ec588ffaa78afa3268b04a,06132809afbe1c781e1b9605452081,67354be5f6f02a74e7661f9529daeb,9aecf32a60e4e0f7f967caecb6c798,403a42b604f504069c41d39c5a4fc2,209f1d1349a473f3eb0b285140c70a,c775ad98eb2cb98e5dd370867bd0cf
...,...,...,...,...,...,...,...,...,...,...
368,a0b55fcae980de79f193f65ca74e80,417164f560d229056b4a7570815c48,568585bc69d537f6e8c68542f84ded,4d90b6cad5f3d54df5e10ec848d47b,a6caf80f7e1afa96600ae6887a56a2,86590a559c1fbf5a853b0250d2e90d,ccffca565b60e96ba17c4e8dc41781,cf9011da196f6aaa5ab407d4aed89f,01c8ee28cc66e7d7801786ba6c8d5f,199cf6d89a2f06fc552ebf86105d7d
369,57a6c27ac59feef428160b724002b5,c582be331080e8790d2899911f3242,d5d0865f3d923e6d753ceae84480b7,c045708b9c384920790737be68d28a,5b09c22ef1a754afad3a1062f4a78e,44b66885feb100106b907262835a46,45be99f14b6b78a5fceb2a7489e8b8,58a923b314735ae6f938d4c51c85f6,637463972c3f9869044cdffaf98c8d,fcdebb544d991f71e3abe0c5b09af2
370,5d9b657b32bff89dedf7c9480ba3e6,41b772a5b18bc724562bfb60f4f3ac,914880359e59dbe7b7e60c4ddc82f4,64679d8550ae26a61ee8ea94ac3fa0,26b3f811a6b9ed75d3cc5198105fa4,e3e5642e76f638dd918e718dfefe3a,ea4f9a22de6d23ff14b645e975230e,87de245b87c3c2ec94b13cef2ae2d1,23ee5640ec588ffaa78afa3268b04a,5b150a1cae009459493bd0fe5c7b3e
371,127c45352d50fad1cd1936b2e21354,961c336b0c2fa04c883c639dfee1ba,514f3ee1f00df05dc3900251972245,2c72ca8794dd4eb8f5ae90ca0dce24,c292588c17f2fb55db17a98774ef07,40b89ab9386c18e49053bf2a65bca8,231b80c884b2f1390e17f09a187889,d6159553e41857fe23df9d3e746284,c0314a446fbef65ce9626b037a70da,bd6707bb04cfb8880a5f783c85be20


In [9]:
movies_id = sessions_df.to_numpy()
movies_id = np.array(movies_id).flatten()
vocab = np.unique(movies_id)
vocab_len = len(vocab)

print(vocab)
print(vocab_len)
print(movies_id.shape)

['001db9af2dcc57d45d653aeb33171e' '00aa9866a9923e9e361f1d2537e538'
 '0138811d772659bbb32c0c32ec6657' '01ab32976170c8eebe4054fd66cc39'
 '01bc55dad49dc1dfa856c85edbf6d1' '01c8ee28cc66e7d7801786ba6c8d5f'
 '02817e267c3e407bc7b207f6ed9f34' '034a4624589199e7f0788ffca6cf5a'
 '03cee3e4cb9a464740ce3c0671c904' '0538507d2f06a33027cf5c78822829'
 '05820afa6837fcafc6e6f4d6391a4c' '05ac7d111c56e0510a9f3b44a33ebc'
 '05b7120042d9597d422c48ba6341b8' '05c92546aa14ea6148088b7c713be4'
 '06030fb2a64f0fc9df81ff76eea9d5' '06132809afbe1c781e1b9605452081'
 '061a59839837a123a916a70947f075' '06cd00fa7fce77cc7c1ad0a64d163e'
 '06d4e0812e40c50f93b3407878e744' '075625b5fbd759bdbc74b3bf2e0091'
 '076fbc4cedbad567d7acf85448be50' '077374b7238b28b0a4aefa135f2f45'
 '07e873c5aac8ffaf97897fbd0dfbf6' '08712d35cf70496630e16b24dae005'
 '0884b214cea2f4f191d459d2ba6c3c' '09887453861c8d4e951ceca579ebfb'
 '0a49f9b4576a63be41cb96908eba5e' '0aaa310d7cc38cf4f884c276f03335'
 '0b5aaa6d4b9941b2f4a31f9480d757' '0be7362fc8c435e83933f8b4336

In [10]:
def vocab_creater(arr):
    word2idx = {}
    idx2word = {}
    for i in range(len(arr)):
        word2idx[arr[i]] = i
        idx2word[i] = arr[i]

    return word2idx, idx2word, len(arr)

In [11]:
word2idx, idx2word, _ = vocab_creater(vocab)

In [12]:
print(_, word2idx)

458 {'001db9af2dcc57d45d653aeb33171e': 0, '00aa9866a9923e9e361f1d2537e538': 1, '0138811d772659bbb32c0c32ec6657': 2, '01ab32976170c8eebe4054fd66cc39': 3, '01bc55dad49dc1dfa856c85edbf6d1': 4, '01c8ee28cc66e7d7801786ba6c8d5f': 5, '02817e267c3e407bc7b207f6ed9f34': 6, '034a4624589199e7f0788ffca6cf5a': 7, '03cee3e4cb9a464740ce3c0671c904': 8, '0538507d2f06a33027cf5c78822829': 9, '05820afa6837fcafc6e6f4d6391a4c': 10, '05ac7d111c56e0510a9f3b44a33ebc': 11, '05b7120042d9597d422c48ba6341b8': 12, '05c92546aa14ea6148088b7c713be4': 13, '06030fb2a64f0fc9df81ff76eea9d5': 14, '06132809afbe1c781e1b9605452081': 15, '061a59839837a123a916a70947f075': 16, '06cd00fa7fce77cc7c1ad0a64d163e': 17, '06d4e0812e40c50f93b3407878e744': 18, '075625b5fbd759bdbc74b3bf2e0091': 19, '076fbc4cedbad567d7acf85448be50': 20, '077374b7238b28b0a4aefa135f2f45': 21, '07e873c5aac8ffaf97897fbd0dfbf6': 22, '08712d35cf70496630e16b24dae005': 23, '0884b214cea2f4f191d459d2ba6c3c': 24, '09887453861c8d4e951ceca579ebfb': 25, '0a49f9b4576a63be

In [13]:
def toArray(smth):
    new = []
    
    for i in range(len(smth)):
        new.append(word2idx[smth[i]])
    
    return new


tokenized = toArray(movies_id)
print(tokenized)

[129, 35, 285, 76, 180, 416, 107, 81, 15, 202, 395, 306, 310, 192, 235, 233, 222, 273, 156, 140, 74, 349, 394, 357, 277, 163, 95, 363, 132, 232, 390, 78, 443, 267, 136, 283, 210, 134, 107, 416, 180, 416, 107, 76, 15, 202, 285, 129, 69, 372, 91, 94, 83, 444, 86, 72, 207, 118, 56, 250, 416, 180, 107, 76, 15, 202, 285, 129, 69, 372, 207, 72, 56, 250, 83, 94, 91, 444, 86, 359, 279, 396, 84, 405, 198, 150, 219, 229, 270, 12, 321, 138, 453, 332, 183, 392, 212, 423, 314, 428, 78, 390, 443, 267, 136, 283, 210, 134, 107, 416, 120, 203, 164, 286, 4, 242, 273, 35, 129, 202, 175, 194, 264, 452, 154, 18, 349, 347, 343, 74, 255, 231, 247, 408, 313, 124, 197, 287, 368, 319, 349, 74, 394, 357, 277, 163, 95, 363, 132, 232, 107, 180, 416, 76, 15, 202, 285, 129, 35, 69, 434, 33, 3, 201, 266, 97, 46, 211, 447, 82, 203, 120, 164, 286, 4, 242, 273, 35, 129, 202, 428, 21, 215, 423, 272, 101, 39, 341, 152, 292, 283, 100, 402, 326, 260, 123, 230, 191, 50, 13, 56, 72, 207, 250, 83, 94, 91, 444, 86, 359, 81, 7, 

In [14]:
dataset = tf.data.Dataset.from_tensor_slices(tokenized[:2500])

In [15]:
seq_length = 10
examples_per_epoch = len(tokenized) // (seq_length + 1)

In [16]:
sequences = dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(seq)

tf.Tensor([129  35 285  76 180 416 107  81  15 202 395], shape=(11,), dtype=int32)


In [17]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 10), (64, 10)), types: (tf.int32, tf.int32)>

In [18]:
for input_example, target_example in dataset.take(1):
    print("Input :", input_example)
    print("Target:", target_example)

Input : tf.Tensor(
[[ 81  76  15 419 103 100 283 403  25 433]
 [120 203 164 286   4 242 273  35 129 202]
 [452 175 194 264  88  18 349 347 343 360]
 [367 379 225 335 273 113 404  30 205 119]
 [279  84 405 198 150 219 229 270 419 134]
 [107  64 388  23  43 404 229 270   6  12]
 [133 246  61 377 170 380 209 430 122 438]
 [ 99 122 430 438  76 430 122 438  19  76]
 [457 415  31  52 213  99 122 430 438  76]
 [357 132 293 268 419 134 416 180 107 251]
 [180 416 107  81  15 202 260 123 288 142]
 [117  89 105 346 254  95 277 163 357  48]
 [270  12 321 138 453 332 183 392 212 423]
 [ 97  46 211 447  82 203 120 164 286   4]
 [ 86 250 294  61 373 188  96  60 120 203]
 [299 405 198 279 236 144 425 336 270 377]
 [250  83  94  91 444  86 359 186 384 118]
 [168 200 367 379 225 335 273  28 325 108]
 [  7 416 350 202 129 180 416 107  81  76]
 [  0 252 170  61 395 380  31 457 415  52]
 [395  76 180 416 107  15 202 285 129  69]
 [357 277 163  95 363 132 232 107 180 416]
 [273  35 129 202 428  21 215 423 2

In [29]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size, activation=activations.softmax)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [30]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 1024

# Number of RNN units
rnn_units = 512
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [31]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 10, 458) # (batch_size, sequence_length, vocab_size)


In [32]:
tf.config.run_functions_eagerly(True)
model.summary()

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  468992    
_________________________________________________________________
gru_1 (GRU)                  multiple                  2362368   
_________________________________________________________________
dense_1 (Dense)              multiple                  234954    
Total params: 3,066,314
Trainable params: 3,066,314
Non-trainable params: 0
_________________________________________________________________


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 10, 458)  # (batch_size, sequence_length, vocab_size)
Mean loss:         6.1254745


In [35]:
model.compile(optimizer='adam', loss=loss, metrics=[tf.keras.metrics.SparseCategoricalAccuracy(
    name='sparse_categorical_accuracy', dtype=None
)])

In [36]:
EPOCHS = 70
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/70
3/3 [==============================] - 1s 209ms/step - loss: 6.1150 - sparse_categorical_accuracy: 0.0740
Epoch 2/70
3/3 [==============================] - 1s 208ms/step - loss: 6.0333 - sparse_categorical_accuracy: 0.3956
Epoch 3/70
3/3 [==============================] - 1s 207ms/step - loss: 5.9220 - sparse_categorical_accuracy: 0.3206
Epoch 4/70
3/3 [==============================] - 1s 209ms/step - loss: 5.7058 - sparse_categorical_accuracy: 0.1753
Epoch 5/70
3/3 [==============================] - 1s 211ms/step - loss: 5.4495 - sparse_categorical_accuracy: 0.1386
Epoch 6/70
3/3 [==============================] - 1s 210ms/step - loss: 5.2379 - sparse_categorical_accuracy: 0.1787
Epoch 7/70
3/3 [==============================] - 1s 210ms/step - loss: 5.0492 - sparse_categorical_accuracy: 0.2189
Epoch 8/70
3/3 [==============================] - 1s 233ms/step - loss: 4.8367 - sparse_categorical_accuracy: 0.2098
Epoch 9/70
3/3 [==============================] - 1s 220ms/step 

In [37]:
train = tf.data.Dataset.from_tensor_slices(tokenized[2500:3730])
sequences = train.batch(seq_length + 1, drop_remainder=True)
train = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000

train = (
    train
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

In [38]:
model.evaluate(train, batch_size=64, return_dict=True)

1/1 [==============================] - 0s 92ms/step - loss: 5.4542 - sparse_categorical_accuracy: 0.3500


{'loss': 5.45421028137207, 'sparse_categorical_accuracy': 0.3499999940395355}

In [39]:
prediction = model.predict([205])

print(prediction.shape)

print(np.argmax(prediction[0]))

(1, 1, 458)
119
